https://stackoverflow.com/questions/24356683/apply-grouped-model-back-onto-data

In [30]:
library(dplyr)
library(broom)
library(tibble)
library(tidyr)
library(purrr)

In [5]:
head(iris)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


In [7]:
models = 
    iris %>%
    group_by(Species) %>%
    do(mod = lm(Sepal.Length ~ Sepal.Width, data = .))

In [9]:
print(models)

Source: local data frame [3 x 2]
Groups: <by row>

# A tibble: 3 x 2
     Species      mod
*     <fctr>   <list>
1     setosa <S3: lm>
2 versicolor <S3: lm>
3  virginica <S3: lm>


In [12]:
x = iris %>% tbl_df
head(x)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


In [16]:
x = iris %>% tbl_df %>% left_join(iris.models, by="Species")
print(x)

# A tibble: 150 x 6
   Sepal.Length Sepal.Width Petal.Length Petal.Width Species      mod
          <dbl>       <dbl>        <dbl>       <dbl>  <fctr>   <list>
 1          5.1         3.5          1.4         0.2  setosa <S3: lm>
 2          4.9         3.0          1.4         0.2  setosa <S3: lm>
 3          4.7         3.2          1.3         0.2  setosa <S3: lm>
 4          4.6         3.1          1.5         0.2  setosa <S3: lm>
 5          5.0         3.6          1.4         0.2  setosa <S3: lm>
 6          5.4         3.9          1.7         0.4  setosa <S3: lm>
 7          4.6         3.4          1.4         0.3  setosa <S3: lm>
 8          5.0         3.4          1.5         0.2  setosa <S3: lm>
 9          4.4         2.9          1.4         0.2  setosa <S3: lm>
10          4.9         3.1          1.5         0.1  setosa <S3: lm>
# ... with 140 more rows


In [18]:
m <- function(df) {
  mod <- lm(Sepal.Length ~ Sepal.Width, data = df)
  pred <- predict(mod, newdata = df["Sepal.Width"])
  data.frame(df, pred)
}

x = iris %>%
    group_by(Species) %>%
    do(m(.))

print(x)

# A tibble: 150 x 6
# Groups:   Species [3]
   Sepal.Length Sepal.Width Petal.Length Petal.Width Species     pred
          <dbl>       <dbl>        <dbl>       <dbl>  <fctr>    <dbl>
 1          5.1         3.5          1.4         0.2  setosa 5.055715
 2          4.9         3.0          1.4         0.2  setosa 4.710470
 3          4.7         3.2          1.3         0.2  setosa 4.848568
 4          4.6         3.1          1.5         0.2  setosa 4.779519
 5          5.0         3.6          1.4         0.2  setosa 5.124764
 6          5.4         3.9          1.7         0.4  setosa 5.331911
 7          4.6         3.4          1.4         0.3  setosa 4.986666
 8          5.0         3.4          1.5         0.2  setosa 4.986666
 9          4.4         2.9          1.4         0.2  setosa 4.641421
10          4.9         3.1          1.5         0.1  setosa 4.779519
# ... with 140 more rows


In [39]:
x = iris %>% 
    nest(-Species) %>%
    inner_join(models, by='Species') %>%
    mutate(pred = map(mod, predict)) %>% 
    unnest(pred, data)
print(x)

# A tibble: 150 x 6
   Species     pred Sepal.Length Sepal.Width Petal.Length Petal.Width
    <fctr>    <dbl>        <dbl>       <dbl>        <dbl>       <dbl>
 1  setosa 5.055715          5.1         3.5          1.4         0.2
 2  setosa 4.710470          4.9         3.0          1.4         0.2
 3  setosa 4.848568          4.7         3.2          1.3         0.2
 4  setosa 4.779519          4.6         3.1          1.5         0.2
 5  setosa 5.124764          5.0         3.6          1.4         0.2
 6  setosa 5.331911          5.4         3.9          1.7         0.4
 7  setosa 4.986666          4.6         3.4          1.4         0.3
 8  setosa 4.986666          5.0         3.4          1.5         0.2
 9  setosa 4.641421          4.4         2.9          1.4         0.2
10  setosa 4.779519          4.9         3.1          1.5         0.1
# ... with 140 more rows
